<a href="https://colab.research.google.com/github/Wayne-wyyking888/qlib__ADARNN_modify/blob/main/(QLIB)ADARNN%E2%80%94%E2%80%94playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1(A). **加载 Qlib**: 股票交易package


#### -> 安装 Qlib & 导入相应的 packages
#### （！）使用GPU （修改-笔记本设定-GPU）


In [ ]:
# Install Qlib in Colab
# !pip install pyqlib
# 这里会报错 但是没事
# !pip install numpy
!pip install --upgrade cython
!pip install --upgrade scipy
!pip install torch==1.11.0+cu102 torchvision==0.12.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html
# GPU stable version

%cd /usr/local/lib/python3.10/dist-packages
!git clone https://github.com/Wayne-wyyking888/qlib__ADARNN_modify.git
%cd qlib__ADARNN_modify

!pip install .
!python setup.py install

#### -> (Optional) 修改了github package，需要先删除再重新安装

In [ ]:
%cd /usr/local/lib/python3.10/dist-packages
%rm -rf qlib__ADARNN_modify
!pip uninstall pyqlib

/usr/local/lib/python3.10/dist-packages
Found existing installation: pyqlib 0.9.3.99
Uninstalling pyqlib-0.9.3.99:
  Would remove:
    /usr/local/bin/qrun
    /usr/local/lib/python3.10/dist-packages/pyqlib-0.9.3.99.dist-info/*
    /usr/local/lib/python3.10/dist-packages/qlib/*
Proceed (Y/n)? Y
  Successfully uninstalled pyqlib-0.9.3.99


### ->(!!) **修改源代码**

### (1) **Modify Handler.py，class Alpha158**：
增加ADARNN 可以使用 Alpha158 的所有fields （in **'data_handler_config'** add **'feature_config'** argument） 以及额外增加features。ADARNN 必须使用一个size； 其他模型如xgboost不一定。 （eg： window = [0,1,4,6,10]， input shape：featuresize X 5（window length））


### (2) **Modify pytorch_adarnn.py：**
A: 增加evalutaion的指标: MSE, L1_loss, RMSE.\
B: 修改data_loader class 和 get_stock_loader 函数； 保证自定义输入feature大小和seq 长度。


In [ ]:
# #现在可以对源码进行修改，修改后需要重新运行以下代码
# %cd /usr/local/lib/python3.10/dist-packages/qlib
# !pip install .

### -> * (!!) **安装完成后重启对话，然后运行下面的**。

In [ ]:
# run this first！！
import qlib
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn

In [ ]:
torch.cuda.is_available()

True

In [ ]:
# Qlib Version
qlib.__version__

'0.9.3.99'

# 2(B). work-flow **上面的一个简单的例子**

### （1）下载 cn 端的股市数据 （删除旧数据 if y）
### **** No data after 2020-09 ****

In [ ]:
%cd ~
!python -m qlib.run.get_data qlib_data --target_dir ~/.qlib/qlib_data/cn_data --region cn

/root
2024-01-04 01:38:38.408 | WARNING  | qlib.tests.data:download_data:79 - The data for the example is collected from Yahoo Finance. Please be aware that the quality of the data might not be perfect. (You can refer to the original data source: https://finance.yahoo.com/lookup.)
2024-01-04 01:38:38.409 | INFO     | qlib.tests.data:download_data:82 - qlib_data_cn_1d_latest.zip downloading......
196549632it [00:08, 22292049.62it/s]                   
2024-01-04 01:38:47.227 | WARNING  | qlib.tests.data:_unzip:104 - will delete the old qlib data directory(features, instruments, calendars, features_cache, dataset_cache): /root/.qlib/qlib_data/cn_data
2024-01-04 01:38:47.227 | INFO     | qlib.tests.data:_unzip:108 - /root/.qlib/qlib_data/cn_data/20240104013837_qlib_data_cn_1d_latest.zip unzipping......
100% 31008/31008 [00:08<00:00, 3789.95it/s]


### （2） **initialize** 以及 导入 packages

In [ ]:
from qlib.constant import REG_CN
#### **** Initialize CN data **** ####
provider_uri = "~/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)
#### ****** ####

[55515:MainThread](2024-01-04 05:13:47,238) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
INFO:qlib.Initialization:default_conf: client.
[55515:MainThread](2024-01-04 05:13:48,722) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
INFO:qlib.Initialization:qlib successfully initialized based on client settings.
[55515:MainThread](2024-01-04 05:13:48,737) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/root/.qlib/qlib_data/cn_data')}
INFO:qlib.Initialization:data_path={'__DEFAULT_FREQ': PosixPath('/root/.qlib/qlib_data/cn_data')}


In [ ]:
#### import lib base for ADARNN Testing model
## (*****) DATA manipulation
from qlib.data import D ## (*****) DATA manipulation
from qlib.data.filter import NameDFilter, ExpressionDFilter
## (***** ------------- *****)

from qlib.contrib.model.gbdt import LGBModel
from qlib.contrib.data.handler import Alpha158, Alpha360
from qlib.utils import init_instance_by_config, flatten_dict
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, SigAnaRecord, PortAnaRecord, HFSignalRecord

ModuleNotFoundError. CatBoostModel are skipped. (optional: maybe installing CatBoostModel can fix it.)


### (3) **数据集的导入和使用例子 （EXAMPLE）**

-> **Instrument**: (代码，市场), 可以使用filter(name, expression) 进行挑选\
-> **Fields**: (因子，预测变量list)\
* 'close': 收盘价；'volume': 成交量
* 'Ref': （引用， #前x天（-value， 后一天））
* 'Mean': （xx， day）均值\

-> ** NO data


In [ ]:
## name 和 expression 过滤器
eg_nameDFilter = NameDFilter(name_rule_re='SH[0-9]{5}[0-9]{1}') # 中间0-9 4位
eg_expressionDFilter = ExpressionDFilter(rule_expression='Rank($high - $low, 1) < 50')
eg_instruments = D.instruments(market='csi300', filter_pipe=[eg_nameDFilter, eg_expressionDFilter])
eg_fields = ['$close', '$volume', 'Ref($vwap, 1)', 'Mean($close, 3)', '(2*Ref($close, 2) - Ref($high, 2) - Ref($low, 2))/(Ref($high, 2)-Ref($low, 2)+1e-12)', 'Corr($close, Log($volume+1), 2)']

#### （I） 正规导入

In [ ]:
## 提取数据，并且导入（test_df)
test_df = D.features(instruments = eg_instruments, fields = eg_fields\
                    ,start_time='2019-01-01', end_time='2020-02-07', freq='day')

#### （II）测试导入

In [ ]:
test_df = D.features(instruments = ['SZ000002', 'SH600123'], fields = eg_fields\
                    ,start_time='2014-01-01', end_time='2021-02-07', freq='day')

In [ ]:
test_df.reset_index()[['instrument','datetime']].groupby('instrument').agg({'instrument': lambda x: x[0:1].values[0]}).iloc[list(range(0,90,3))].instrument.values

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['SH600000', 'SH600010', 'SH600016', 'SH600023', 'SH600028',
       'SH600031', 'SH600048', 'SH600066', 'SH600089', 'SH600109',
       'SH600118', 'SH600170', 'SH600183', 'SH600208', 'SH600233',
       'SH600297', 'SH600332', 'SH600346', 'SH600369', 'SH600390',
       'SH600415', 'SH600482', 'SH600498', 'SH600519', 'SH600547',
       'SH600570', 'SH600588', 'SH600655', 'SH600674', 'SH600703'],
      dtype=object)

In [ ]:
D.features(instruments = ['SH600011'], fields = eg_fields\
                    ,start_time='2020-09-01', end_time='2020-10-20', freq='day')

$close      $volume  Ref($vwap, 1)  Mean($close, 3)  \
instrument datetime                                                            
SH600011   2020-09-01  2.264353  204091792.0            NaN         2.184359   
           2020-09-02  2.237258  105607056.0            NaN         2.211454   
           2020-09-03  2.206293  116122936.0            NaN         2.235968   
           2020-09-04  2.144362   99370680.0            NaN         2.195971   
           2020-09-07  2.109525   74148264.0            NaN         2.153393   
           2020-09-08  2.190810   99497784.0            NaN         2.148232   
           2020-09-09  2.202422   72951832.0            NaN         2.167586   
           2020-09-10  2.275965  122211632.0            NaN         2.223066   
           2020-09-11  2.256612   80619920.0            NaN         2.245000   
           2020-09-14  2.264353   94642200.0            NaN         2.265643   
           2020-09-15  2.217905   54641836.0            NaN         2.246290   
           2020-09-16  2.214034   43993844.0            NaN         2.232097   
           2020-09-17  2.163715   36612816.0            NaN         2.198551   
           2020-09-18  2.245000   55275136.0            NaN         2.207583   
           2020-09-21  2.163715   78055360.0            NaN         2.190810   
           2020-09-22  2.136620   35124732.0            NaN         2.181778   
           2020-09-23  2.113396   34181868.0            NaN         2.137911   
           2020-09-24  2.070818   44475248.0            NaN         2.106945   
           2020-09-25  2.113396   36318820.0            NaN         2.099204   

                       (2*Ref($close, 2) - Ref($high, 2) - Ref($low, 2))/(Ref($high, 2)-Ref($low, 2)+1e-12)  \
instrument datetime                                                                                           
SH600011   2020-09-01                                           0.655173                                      
           2020-09-02                                          -0.846152                                      
           2020-09-03                                           0.952381                                      
           2020-09-04                                           0.300000                                      
           2020-09-07                                           0.034481                                      
           2020-09-08                                          -0.806450                                      
           2020-09-09                                          -0.578943                                      
           2020-09-10                                           0.405403                                      
           2020-09-11                                           0.124997                                      
           2020-09-14                                           0.500003                                      
           2020-09-15                                           0.739128                                      
           2020-09-16                                           0.555558                                      
           2020-09-17                                          -0.473679                                      
           2020-09-18                                           0.499995                                      
           2020-09-21                                          -0.733330                                      
           2020-09-22                                           1.000000                                      
           2020-09-23                                          -0.703706                                      
           2020-09-24                                          -0.272730                                      
           2020-09-25                                          -0.599998                                      

            

# 3(C). 测试模型 ADARNN: **LABEL is REturn**
* Output must be **one-dim**
* **Alpha 360**: 6 features; **Alpha 158** (*impossible to use* ): 20+ features
* *? data handler*
1.   **Instruments**: size >= 2; Individually train models.
（***Q： 每个股票是一个模型吗？***
（
2.   **Fields：Item  'infer_processors', 'learn_processors' can be manually setup**! \
（***Q：Alpha360 的参数是哪些？ 可以自己加features吗？***）\
（Ans： qilb.Utils.Handler.py：ALPHA360(adaRNN) 固定60维seq， 6维features（都是normalized）, \
[注意]： 'data_loader' 要进入源代码更改 ）
3. **output**   （***Q：输出的score到底是什么？输出的维度可以调整吗？***）\
（Ans： 输出一定是一维，就是正常的prediction）
[注意]: 'pytorch_adarnn.py' 添加parameter和输出维度
4.  **LABEL**: **Self-specified label**.



### -> **Model SETUP**

In [ ]:
market = "csi300"
eg_fields = ['$close', '$volume', 'Ref($close, 1)', 'Mean($close, 3)', '$high-$low']
eg_instruments = ['SH600000', 'SH600010', 'SH600016', 'SH600023', 'SH600028',
       'SH600031', 'SH600048', 'SH600066', 'SH600089', 'SH600109',
       'SH600118', 'SH600170', 'SH600183', 'SH600208', 'SH600233',
       'SH600297', 'SH600332', 'SH600346', 'SH600369', 'SH600390',
       'SH600415', 'SH600482', 'SH600498', 'SH600519', 'SH600547',
       'SH600570', 'SH600588', 'SH600655', 'SH600674', 'SH600703']
eg_fields = ['$close', '$volume', 'Ref($close, 1)', 'Mean($close, 3)', '$high', '$low']
benchmark = "SH000300"
data_handler_config = {
    "start_time": "2017-01-01",
    "end_time": "2020-09-10",
    "fit_start_time": "2017-06-01",
    "fit_end_time": "2018-12-31",
    ### NEW INPUTS ###
    "feature_config" : {
            'window.size' : 30, # window size by default
            'kbar': {'windows': [i for i in range(50)],
                     'features': ["KMID2", "KSFT", "KLOW2", "KUP", "KLEN"]}, # whether to use some hard-code kbar features
            # whether to use raw price features
            'price': {'windows': [i for i in range(50)],
                      'features': ["OPEN", "HIGH", "LOW", "CLOSE", 'VWAP']}, # which price field (ratio) to use
            # whether to use raw volume (ratio) features
            'volume': {'windows': [i for i in range(50)]},
             # whether to use rolling operator based features
            # window should not include 0 !!
            'rolling': {'windows': [i for i in range(3,103,2)],
                        'features': ["QTLU", "CORR", "VSTD", "VMA", "BETA",
                              "ROC", "MA", "RANK", "IMIN", "IMXD",
                              "RESI", "RSV", "CNTN", "SUMD", "VSUMD",
                              "STD", "SUMP", "CNTD", "WVMA"]},
            #  # additional features to be used; must have the same window size
            # # MUST be a price object! (name will be, eg, OPEN_3_pure, 'pure' is for original feature without ratios)
            # 'additional': {'windows': [i for i in range(60)],
            #                'features': ["OPEN", "HIGH", "LOW", "CLOSE", 'VWAP']}
    },
    ### REAL ###
    # "instruments": eg_instruments, # 股票池子
    # "label": ["Ref($close, -2) / Ref($close, -1) - 1"], # response: 后1天收益率 (BENCHMARK)
    ### 测试 ###
    "instruments": ['SH600297', 'SH600332', 'SH600346', 'SH600369', 'SH600390',
        'SH600415', 'SH600482', 'SH600498', 'SH600519', 'SH600547',
       'SH600570', 'SH600588', 'SH600655', 'SH600674', 'SH600703'],
    "label": ["Ref($close, -2) / Ref($close, -1) - 1"], # response: 后1天收益率 (BENCHMARK)
    ## Inference processer
    'infer_processors':
    [
    {"class": "ProcessInf", "kwargs": {}},
    {"class": "ZScoreNorm", "kwargs": {}},
    {"class": "Fillna", "kwargs": {}},
        # {
        #     "class" : "FilterCol", # features
        #     "kwargs":{
        #     "fields_group": "feature",
        #     # "col_list": ["RESI5", "WVMA5", "RSQR5", "KLEN", "RSQR10", "CORR5", "CORD5", "CORR10",
        #     #                 "ROC60", "RESI10", "VSTD5", "RSQR60", "CORR60", "WVMA60", "STD5",
        #     #                 "RSQR20", "CORD60", "CORD10", "CORR20", "KLOW"]
        #     "col_list": ["RESI5", "WVMA5", "RSQR5", "KLEN", "RSQR10", "CORR5"]}
        # },
        # {"class": "ProcessInf", "kwargs": {}},
        # {
        #     'class': 'RobustZScoreNorm', # normalization
        #     "kwargs":{
        #     "fields_group": "feature", 'clip_outlier': True}
        # },
        # {
        #      "class" : "CSZFillna", # deal with na
        #      "kwargs":{"fields_group": "feature"}
        # }
    ],
    # learner_processor
    'learn_processors': [
    {"class": "DropnaLabel"},
    {"class": "CSZScoreNorm", "kwargs": {"fields_group": "label"}},
]

     #[
    #     {
    #         "class" : "DropnaLabel"
    #     },
    #     {
    #         "class": "CSRankNorm",
    #         "kwargs": {"fields_group": "label"}
    #     },
    #     {
    #         "class": "CSZScoreNorm",
    #         "kwargs": {"fields_group": "label"}
    #     }
    # ]

} # handle data (don't have to extract， configuration file)

task = {
    "model": {
        "class": "ADARNN",
        "module_path": "qlib.contrib.model.pytorch_adarnn",
        "kwargs": {
            'd_feat': 30,
            'len_seq': 50, # fixed sequence length
            'hidden_size': 64,
            'num_layers': 5,
            'dropout': 0.5,
            'n_epochs': 40,
            'lr': 1e-3, #
            'early_stop': 20,
            'batch_size': 80, # small
            "optimizer": "adam", # ADAM
            'metric': 'loss',
            'loss': 'mse',
            'n_splits': 3,
            'GPU': 0
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha158",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": ("2017-06-01", "2018-12-31"),
                "valid": ("2019-01-01", "2019-06-30"),
                "test": ("2019-07-01", "2020-09-01"),
            },
        },
    },
}

# model initiaiton
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])


[55515:MainThread](2024-01-04 06:16:50,167) INFO - qlib.ADARNN - [pytorch_adarnn.py:64] - ADARNN pytorch version...
INFO:qlib.ADARNN:ADARNN pytorch version...
[55515:MainThread](2024-01-04 06:16:50,172) INFO - qlib.ADARNN - [pytorch_adarnn.py:89] - ADARNN parameters setting:
d_feat : 30
d_seq: 50
hidden_size : 64
num_layers : 5
dropout : 0.5
n_epochs : 40
lr : 0.001
metric : loss
batch_size : 80
early_stop : 20
optimizer : adam
loss_type : mse
n_splits: 3
visible_GPU : 0
use_GPU : True
seed : None
INFO:qlib.ADARNN:ADARNN parameters setting:
d_feat : 30
d_seq: 50
hidden_size : 64
num_layers : 5
dropout : 0.5
n_epochs : 40
lr : 0.001
metric : loss
batch_size : 80
early_stop : 20
optimizer : adam
loss_type : mse
n_splits: 3
visible_GPU : 0
use_GPU : True
seed : None
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and nu

### -> **Evaluation Results** (pred 和 IC， ICIR.. )

* signalrecord (**prediction value**) 产生 'pred.pkl' 和 'label.pkl', 两个数据
* HFSignalRecord (record **IC, ICIR, RANKIC, RANKICIR & long/short term precision**)

In [ ]:
# start exp
with R.start(experiment_name="workflow"):
    # train
    R.log_params(**flatten_dict(task))
    model.fit(dataset)

    # prediction
    recorder = R.get_recorder()
    # (1) signalrecord (prediction value)
    # save as 'pred.pkl' (for prediction score), 'label.pkl' (original label)
    sr_example = SignalRecord(model, dataset, recorder) # generate prediction results
    sr_example.generate() # Real Save

    # (2) HFSignalRecord (Criterion Values: IC, ICIR ...)
    sr_ic_example = HFSignalRecord(recorder) # generate IC, ICIR, rankIC, rankICIR, (long/short term) precision
    sr_ic_example.generate()

[55515:MainThread](2024-01-04 06:50:47,766) INFO - qlib.workflow - [exp.py:258] - Experiment 1 starts running ...
INFO:qlib.workflow:Experiment 1 starts running ...
[55515:MainThread](2024-01-04 06:50:47,784) INFO - qlib.workflow - [recorder.py:341] - Recorder e5c0273b1c7046c8b6a30a426e7d6fb5 starts running under Experiment 1 ...
INFO:qlib.workflow:Recorder e5c0273b1c7046c8b6a30a426e7d6fb5 starts running under Experiment 1 ...
[55515:MainThread](2024-01-04 06:50:47,795) INFO - qlib.workflow - [recorder.py:374] - Fail to log the uncommitted code of $CWD(/content) when run git diff.
INFO:qlib.workflow:Fail to log the uncommitted code of $CWD(/content) when run git diff.
[55515:MainThread](2024-01-04 06:50:47,805) INFO - qlib.workflow - [recorder.py:374] - Fail to log the uncommitted code of $CWD(/content) when run git status.
INFO:qlib.workflow:Fail to log the uncommitted code of $CWD(/content) when run git status.
[55515:MainThread](2024-01-04 06:50:47,817) INFO - qlib.workflow - [recor

'The following are prediction results of the ADARNN model.'
                          score
datetime   instrument          
2019-07-01 SH600297   -0.059741
           SH600332   -0.053467
           SH600346    0.154242
           SH600369    0.054522
           SH600390    0.301240


/usr/local/lib/python3.10/dist-packages/qlib/contrib/eva/alpha.py:59: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  l_dom = groupll.apply(lambda x: x > 0)
/usr/local/lib/python3.10/dist-packages/qlib/contrib/eva/alpha.py:63: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  s_dom = groups.apply(lamb

{'IC': -0.0024026093663152004,
 'ICIR': -0.007936160117137402,
 'Long precision': 0.45601851851851843,
 'Long-Short Average Return': -6.1963496e-05,
 'Long-Short Average Sharpe': -0.0067841685,
 'Rank IC': 0.010010804311481781,
 'Rank ICIR': 0.03576245934546877,
 'Short precision': 0.5324074074074073}


In [ ]:
import mlflow
mlflow.end_run()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
sr_ic_example.generate()

In [ ]:
pred_df = recorder.load_object('pred.pkl') # prediction Result
raw_label_df = recorder.load_object('label.pkl') # raw data label

### -> **Model Check**

 Compare Results of prediction (score) and raw_label (return)

In [ ]:
pd.concat([pred_df.reset_index()[pred_df.reset_index().instrument.isin(['SH600297', 'SH600482', 'SH600390'])].set_index(['datetime','instrument']),
raw_label_df.reset_index()[raw_label_df.reset_index().instrument.isin(['SH600297', 'SH600482', 'SH600390'])].set_index(['datetime','instrument'])],axis = 1).head(30)


score  Ref($close, -2) / Ref($close, -1) - 1
datetime   instrument                                                 
2019-07-01 SH600297    0.471977                              -0.013187
           SH600390    0.857801                              -0.015306
           SH600482   -0.420144                               0.045527
2019-07-02 SH600297    0.420112                               0.000000
           SH600390    0.326989                              -0.010363
           SH600482    0.023562                              -0.016660
2019-07-03 SH600297    0.254117                              -0.006681
           SH600390   -0.239554                               0.001048
           SH600482   -1.222960                              -0.036581
2019-07-04 SH600297    0.022313                              -0.049327
           SH600390   -0.396671                              -0.042887
           SH600482   -0.620392                              -0.036771
2019-07-05 SH600297    0.226371                              -0.004717
           SH600390    0.199744                               0.000000
           SH600482   -1.026572                               0.009544
2019-07-08 SH600297    1.513613                               0.004739
           SH600390    1.034408                              -0.001092
           SH600482   -0.738860                               0.001233
2019-07-09 SH600297    1.575651                              -0.010651
           SH600390    0.714865                               0.001094
           SH600482   -0.899279                              -0.004105
2019-07-10 SH600297    0.559308                              -0.009569
           SH600390   -0.855264                              -0.002186
           SH600482    1.490273                              -0.015664
2019-07-11 SH600297   -0.063041                               0.014493
           SH600390    1.181056                               0.013144
           SH600482    2.768078                               0.030570
2019-07-12 SH600297    0.300988                               0.019048
           SH600390    1.840280                              -0.006487
           SH600482    2.502244                              -0.012190

In [ ]:
pred_df.loc[('2019-07-30', 'SH600297'),], raw_label_df.head(20),
# D.features(instruments = ['SH600000', 'SH600010', 'SH600011', 'SH600030', 'SH600031',
#        'SH600050', 'SH600060', 'SH600062', 'SH600100', 'SH600111',
#        'SH600123', 'SH600143', 'SH600150', 'SH600153', 'SH600160']], fields = ['$close', '$high', '$close/Ref($close,1)-1']\
#                     ,start_time='2016-01-01', end_time='2016-01-11', freq='day')

(score   -0.403678
 Name: (2019-07-30 00:00:00, SH600297), dtype: float32,
                        Ref($close, -2) / Ref($close, -1) - 1
 datetime   instrument                                       
 2019-07-01 SH600297                                -0.013187
            SH600332                                 0.005697
            SH600346                                -0.027822
            SH600369                                -0.013861
            SH600390                                -0.015306
            SH600415                                 0.000000
            SH600482                                 0.045527
            SH600498                                -0.019014
            SH600519                                -0.036195
            SH600547                                 0.034089
            SH600570                                -0.026218
            SH600588                                -0.029007
            SH600655                                 0.00

# 4(D). 模型：(**DEPRECATED**) LABEL is ***Price*** (close, high, low)
* Output must be **one-dim**




### -> **Model Setup**

In [ ]:
market = "csi300"
eg_fields = ['$close', '$volume', 'Ref($close, 1)', 'Mean($close, 3)', '$high-$low']
eg_instruments = ['SH600000', 'SH600010', 'SH600016', 'SH600023', 'SH600028',
       'SH600031', 'SH600048', 'SH600066', 'SH600089', 'SH600109',
       'SH600118', 'SH600170', 'SH600183', 'SH600208', 'SH600233',
       'SH600297', 'SH600332', 'SH600346', 'SH600369', 'SH600390',
       'SH600415', 'SH600482', 'SH600498', 'SH600519', 'SH600547',
       'SH600570', 'SH600588', 'SH600655', 'SH600674', 'SH600703']
eg_fields = ['$close', '$volume', 'Ref($close, 1)', 'Mean($close, 3)', '$high', '$low']
benchmark = "SH000300"

data_handler_config = {
    "start_time": "2017-01-01",
    "end_time": "2020-09-10",
    "fit_start_time": "2017-06-01",
    "fit_end_time": "2018-12-31",
    ### NEW INPUTS ###
    "feature_config" : {
            'window.size' : 60, # window size by default
            'kbar': {'windows': [i for i in range(60)],
                     'features': ["KMID2", "KSFT", "KLOW2", "KUP", "KLEN"]}, # whether to use some hard-code kbar features
            # whether to use raw price features
            'price': {'windows': [i for i in range(60)],
                      'features': ["OPEN", "HIGH", "LOW", "CLOSE", 'VWAP']}, # which price field (ratio) to use
            # whether to use raw volume (ratio) features
            'volume': {'windows': [i for i in range(60)]},
             # whether to use rolling operator based features
            # window should not include 0 !!
            'rolling': {'windows': [i for i in range(3,123,2)],
                        'features': ["QTLU", "CORR", "VSTD", "VMA", "BETA", "RESI", "RSV", "CNTN", "SUMD", "VSUMD"]},
            #  # additional features to be used; must have the same window size
            # # MUST be a price object! (name will be, eg, OPEN_3_pure, 'pure' is for original feature without ratios)
            'additional': {'windows': [i for i in range(60)],
                           'features': ["OPEN", "HIGH", "LOW", "CLOSE", 'VWAP']}
    },
    ### REAL ###
    # "instruments": eg_instruments, # 股票池子
    # "label": ["Ref($close, -2) / Ref($close, -1) - 1"], # response: 后1天收益率 (BENCHMARK)
    ### 测试 ###
    "instruments": ['SH600415', 'SH600482', 'SH600498', 'SH600519', 'SH600547',
       'SH600570', 'SH600588', 'SH600655', 'SH600674', 'SH600703'],
    "label": ["Ref($close, -1)"], # response: 后一天收盘价
    ## Inference processer
    'infer_processors':
    [
    {"class": "ProcessInf", "kwargs": {}},
    {"class": "ZScoreNorm", "kwargs": {"fields_group": "feature"}},
    {"class": "Fillna", "kwargs": {}},
        # {
        #     "class" : "FilterCol", # features
        #     "kwargs":{
        #     "fields_group": "feature",
        #     # "col_list": ["RESI5", "WVMA5", "RSQR5", "KLEN", "RSQR10", "CORR5", "CORD5", "CORR10",
        #     #                 "ROC60", "RESI10", "VSTD5", "RSQR60", "CORR60", "WVMA60", "STD5",
        #     #                 "RSQR20", "CORD60", "CORD10", "CORR20", "KLOW"]
        #     "col_list": ["RESI5", "WVMA5", "RSQR5", "KLEN", "RSQR10", "CORR5"]}
        # },
        # {"class": "ProcessInf", "kwargs": {}},
        # {
        #     'class': 'RobustZScoreNorm', # normalization
        #     "kwargs":{
        #     "fields_group": "feature", 'clip_outlier': True}
        # },
        # {
        #      "class" : "CSZFillna", # deal with na
        #      "kwargs":{"fields_group": "feature"}
        # }
    ],
    # learner_processor
    'learn_processors': [
    {"class": "DropnaLabel"},
#    {"class": "CSZScoreNorm", "kwargs": {"fields_group": "label"}},
]

     #[
    #     {
    #         "class" : "DropnaLabel"
    #     },
    #     {
    #         "class": "CSRankNorm",
    #         "kwargs": {"fields_group": "label"}
    #     },
    #     {
    #         "class": "CSZScoreNorm",
    #         "kwargs": {"fields_group": "label"}
    #     }
    # ]

} # handle data (don't have to extract， configuration file)

task = {
    "model": {
        "class": "ADARNN",
        "module_path": "qlib.contrib.model.pytorch_adarnn",
        "kwargs": {
            'd_feat': 26,
            'len_seq': 60, # fixed sequence length
            'hidden_size': 64,
            'num_layers': 5,
            'dropout': 0.5,
            'n_epochs': 100,
            'lr': 0.5*1e-3, # Adam optimizer
            'early_stop': 20,
            'batch_size': 100, # small
            'metric': 'loss',
            'loss': 'mse',
            'n_splits': 5,
            'GPU': 0
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha158",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": ("2017-06-01", "2018-12-31"),
                "valid": ("2019-01-01", "2019-06-30"),
                "test": ("2019-07-01", "2020-09-01"),
            },
        },
    },
}

# model initiaiton
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])


[19424:MainThread](2024-01-03 09:59:18,874) INFO - qlib.ADARNN - [pytorch_adarnn.py:64] - ADARNN pytorch version...
INFO:qlib.ADARNN:ADARNN pytorch version...
[19424:MainThread](2024-01-03 09:59:18,881) INFO - qlib.ADARNN - [pytorch_adarnn.py:89] - ADARNN parameters setting:
d_feat : 26
d_seq: 60
hidden_size : 64
num_layers : 5
dropout : 0.5
n_epochs : 100
lr : 0.0005
metric : loss
batch_size : 100
early_stop : 20
optimizer : adam
loss_type : mse
n_splits: 5
visible_GPU : 0
use_GPU : False
seed : None
INFO:qlib.ADARNN:ADARNN parameters setting:
d_feat : 26
d_seq: 60
hidden_size : 64
num_layers : 5
dropout : 0.5
n_epochs : 100
lr : 0.0005
metric : loss
batch_size : 100
early_stop : 20
optimizer : adam
loss_type : mse
n_splits: 5
visible_GPU : 0
use_GPU : False
seed : None
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.

KeyboardInterrupt: ignored

### -> **Evaluation Results** (pred 和 IC， ICIR.. )

* signalrecord (**prediction value**) 产生 'pred.pkl' 和 'label.pkl', 两个数据
* HFSignalRecord (record **IC, ICIR, RANKIC, RANKICIR & long/short term precision**)

In [ ]:
# start exp
with R.start(experiment_name="workflow"):
    # train
    R.log_params(**flatten_dict(task))
    model.fit(dataset)

    # prediction
    recorder = R.get_recorder()
    # (1) signalrecord (prediction value)
    # save as 'pred.pkl' (for prediction score), 'label.pkl' (original label)
    sr_example = SignalRecord(model, dataset, recorder) # generate prediction results
    sr_example.generate() # Real Save

    # (2) HFSignalRecord (Criterion Values: IC, ICIR ...)
    sr_ic_example = HFSignalRecord(recorder) # generate IC, ICIR, rankIC, rankICIR, (long/short term) precision
    sr_ic_example.generate()

In [ ]:
sr_ic_example.generate()

In [ ]:
pred_df = recorder.load_object('pred.pkl') # prediction Result
raw_label_df = recorder.load_object('label.pkl') # raw data label

In [ ]:
pred_df.head(20), raw_label_df.head(20),
# D.features(instruments = ['SH600000', 'SH600010', 'SH600011', 'SH600030', 'SH600031',
#        'SH600050', 'SH600060', 'SH600062', 'SH600100', 'SH600111',
#        'SH600123', 'SH600143', 'SH600150', 'SH600153', 'SH600160']], fields = ['$close', '$high', '$close/Ref($close,1)-1']\
#                     ,start_time='2016-01-01', end_time='2016-01-11', freq='day')

#### -> 寻找 Ada RNN

In [ ]:
qlib.utils

In [ ]:
qlib.utils